In [6]:
import time
import json
import tweepy 
import pandas as pd

from credentials import (
    BEARER_TOKEN,
    API_KEY,
    API_SECRET,
    ACCESS_TOKEN,
    ACCESS_TOKEN_SECRET,
)

In [2]:
# Set up your Twitter API credentials
consumer_key = API_KEY
consumer_secret = API_SECRET
access_token = ACCESS_TOKEN
access_token_secret = ACCESS_TOKEN_SECRET

In [3]:
# Authenticate with Twitter API
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

# Create a Tweepy API object
api = tweepy.API(auth)

In [4]:
# Specify the username and the tweet identifier (or URL)
username = "OpenAI"
tweet_url = "https://twitter.com/petrogustavo/status/1666850099031420930"  # Replace with the desired tweet URL

In [5]:
# Extract the tweet_id from the tweet URL
tweet_id = tweet_url.split("/")[-1]

# Alternatively, if you have the tweet ID directly, you can assign it directly
# tweet_id = "123456789"

# Retrieve the tweet object using the tweet ID
tweet = api.get_status(tweet_id)

# Extract the tweet_id from the tweet object
tweet_id = tweet.id_str

# Print the tweet_id
print("Tweet ID:", tweet_id)

Tweet ID: 1666850099031420930


In [9]:
# Streaming Class

class MyStream(tweepy.StreamingClient):
    def __init__(self, auth, tweet_id, csv_file):
        super().__init__(auth)
        self.tweet_id = tweet_id
        self.csv_file = csv_file
        self.number_of_tweets = 0

    def on_connect(self):
        print("Connected")

    def on_data(self, raw_data):
        try:
            tweet = json.loads(raw_data)["data"]
            if self.is_reply_to_tweet(tweet):
                self.process_tweet(tweet)
                self.number_of_tweets += 1
                if self.number_of_tweets % 100 == 0:
                    self.save_to_csv()
            time.sleep(0.2)
        except Exception as e:
            print(e)

    def is_reply_to_tweet(self, tweet):
        referenced_tweets = tweet.get("referenced_tweets")
        if referenced_tweets:
            for referenced_tweet in referenced_tweets:
                if referenced_tweet.get("type") == "replied_to" and referenced_tweet.get("id") == self.tweet_id:
                    return True
        return False

    def process_tweet(self, tweet):
        tweet_data = {
            "text": tweet.get("text"),
            "public_metrics": tweet.get("public_metrics"),
            "annotations": tweet.get("annotations"),
            "entities": tweet.get("entities"),
            "referenced_tweets": tweet.get("referenced_tweets"),
            "author_id": tweet.get("author_id"),
            "created_at": tweet.get("created_at")
        }

        # Add your custom logic here to handle the tweet data

        print(tweet_data)  # Print the tweet data as an example

    def save_to_csv(self):
        df = pd.DataFrame(self.tweets)
        df.to_csv(self.csv_file, index=False, mode="a", header=not self.number_of_tweets)

    def on_disconnect(self):
        print("Disconnected")


In [11]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

stream = MyStream(auth, tweet_id=tweet_id, csv_file="stream_data.csv")  # Replace with the desired tweet ID and CSV filename/path
stream.filter()

Stream encountered HTTP error: 401
HTTP error response text: {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}
Stream encountered HTTP error: 401
HTTP error response text: {
  "title": "Unauthorized",
  "type": "about:blank",
  "status": 401,
  "detail": "Unauthorized"
}


Disconnected


KeyboardInterrupt: 